## Day 7: Recursive Circus

http://adventofcode.com/2017/day/7

### Part 1

First sort out which disks are carrying which other disks. Record the weight as well, that's likely to be useful. The Haskellish parsing code is a bit over the top but I find it easier to read than nested split statements with hacks to remove punctuation or, worse, a regular expression.

In [1]:
import parsy
from collections import namedtuple

Disk = namedtuple('Disk', 'weight subtowers')

chew_whitespace = lambda p: parsy.whitespace.optional() >> p << parsy.whitespace.optional()
bracketed = lambda p: parsy.string('(') >> p << parsy.string(')')
comma = parsy.string(',')
arrow = parsy.string('->')
number = parsy.digit.at_least(1).concat().map(int)
# It's not specified but I'm assuming the names are alphanumerical
p_name = (parsy.letter | parsy.digit).at_least(1).concat()
p_weight = chew_whitespace(bracketed(number))
p_subtowers = chew_whitespace(arrow).optional() >> p_name.sep_by(chew_whitespace(comma))

@parsy.generate
def p_disk():
    disk_name = yield p_name
    weight = yield p_weight
    subtowers = yield p_subtowers
    return (disk_name, Disk(weight, subtowers))

p_disk.parse('fwft (72) -> ktlj, cntj, xhth')

('fwft', Disk(weight=72, subtowers=['ktlj', 'cntj', 'xhth']))

In [2]:
p_disk.parse('pbga (66)')

('pbga', Disk(weight=66, subtowers=[]))

Looks OK.

In [3]:
with open('input', 'r') as f:
    disks = {name: disk for name, disk in (p_disk.parse(line.strip()) for line in f)}

The bottom disk is the one which isn't contained in a subtower of another disk.

In [4]:
set(disks) - set().union(*[set(disks[d].subtowers) for d in disks])

{'dgoocsw'}

### Part 2

To find the unbalancing disk, we need to calculate the total weight of the subtowers on each disk. This can be done dynamically by starting at the top and working down the levels.

First build a stack from the bottom disk up. Then going from the top level down calculate the total weight at each disk, this way the weights for the subtowers will already have been calculated. When a disk has subtowers with differing weights, the subtower with the different weight to the others is the unbalanced disk that needs its weight correcting.

In [5]:
from collections import deque
from collections import defaultdict

disk_stack = []
disks_to_add = deque(['dgoocsw'])

while disks_to_add:
    d = disks_to_add.popleft()
    disks_to_add.extend(disks[d].subtowers)
    disk_stack.append(d)

def all_equal(collection):
    l = list(collection)
    return not l or all(x == l[0] for x in l[1:])

subtower_weights = {}

while disk_stack:
    d = disk_stack.pop()
    weights_at_d = {st: subtower_weights[st] for st in disks[d].subtowers}
    
    # Are all subtowers balanced?
    if all_equal(weights_at_d.values()):
    # If so, calculate the total weight at this disk.
        subtower_weights[d] = disks[d].weight + sum(weights_at_d.values())
    else:
    # Otherwise find the subtower with a different weight from the others.
    # (I'll assume there are more than two otherwise the solution won't be unique.)
        weight_instances = defaultdict(list)
        for st in weights_at_d:
            weight_instances[subtower_weights[st]].append(st)
        wrong_weight = min(weight_instances, key=lambda k: len(weight_instances[k]))
        wrong_disk = weight_instances[wrong_weight][0]
        right_weight = max(weight_instances, key=lambda k: len(weight_instances[k]))
        weight_correction = right_weight - wrong_weight
        corrected_weight = disks[wrong_disk].weight + weight_correction
        print(corrected_weight)
        break

1275


Right answer, but horribly inelegant code. 